In [1]:
import pandas as pd
import numpy as np
import os 
import pickle
#from fuzzywuzzy import process, fuzz
from rapidfuzz import process, utils, fuzz
from collections import Counter
from tqdm import tqdm
pd.options.display.max_columns = None

# checking the dataset's data
Checking the data from the [given](https://www.kaggle.com/datasets/andrewmvd/spotify-playlists?select=spotify_dataset.csv) database.

In [2]:
data = pd.read_csv('../data/charts.csv', sep = ',', on_bad_lines='skip')
print(data.shape)
data.head()

(26173514, 9)


,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0


In [3]:
data[['region', 'chart']].value_counts()

region         chart  
Brazil         top200     364516
Argentina      top200     364385
United States  top200     364184
Austria        top200     363925
Australia      top200     362189
                           ...  
India          viral50     51688
Ukraine        viral50     26778
Russia         viral50     26069
South Korea    viral50     16700
Luxembourg     top200       8962
Length: 139, dtype: int64

For my analysis, I will consider one of the regions given by the dataset. In this case, Spain.

In [4]:
data[data['region'] == 'United States'][['region', 'chart']].value_counts()

region         chart  
United States  top200     364184
               viral50     90873
dtype: int64

In [5]:
data_usa = data[(data['region'] == 'United States')]

check if data has duplicated values

In [6]:
data_usa.columns

Index(['title', 'rank', 'date', 'artist', 'url', 'region', 'chart', 'trend',
       'streams'],
      dtype='object')

In [7]:
data_usa.duplicated().sum()

0

In [8]:
nulos = (data_usa.isnull().sum() / data_usa.shape[0] * 100).reset_index()
nulos = nulos[nulos[0] > 0]
nulos.columns = ["columnas", "%_nulos"]
nulos

,columnas,%_nulos
8,streams,19.969586


No need to remove columns, all have values.

In [9]:
data_usa.head()

,title,rank,date,artist,url,region,chart,trend,streams
8246,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.0
8247,Fake Love,2,2017-01-01,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,United States,top200,SAME_POSITION,1180074.0
8248,Starboy,3,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0
8249,Closer,4,2017-01-01,"The Chainsmokers, Halsey",https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,United States,top200,SAME_POSITION,1010492.0
8250,Black Beatles,5,2017-01-01,"Rae Sremmurd, Gucci Mane",https://open.spotify.com/track/6fujklziTHa8uoM...,United States,top200,SAME_POSITION,874289.0


# Export
I will export the whole dataset and generate a dataset with the tracks and artist without duplicated rows.

In [10]:
with open('../data/pickle/dataset_usa_base.pickle', 'wb') as data:
    pickle.dump(data_usa, data)

# Generate clean artist - track data
I will now get a unique list from the track - artist relation, in order to make api calls.

In [11]:
data_usa.head()

,title,rank,date,artist,url,region,chart,trend,streams
8246,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.0
8247,Fake Love,2,2017-01-01,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,United States,top200,SAME_POSITION,1180074.0
8248,Starboy,3,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0
8249,Closer,4,2017-01-01,"The Chainsmokers, Halsey",https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,United States,top200,SAME_POSITION,1010492.0
8250,Black Beatles,5,2017-01-01,"Rae Sremmurd, Gucci Mane",https://open.spotify.com/track/6fujklziTHa8uoM...,United States,top200,SAME_POSITION,874289.0


In [12]:
data_usa = data_usa[['title', 'artist', 'url']]
print(data_usa.shape)
data_usa.head()

(455057, 3)


,title,artist,url
8246,Bad and Boujee (feat. Lil Uzi Vert),Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...
8247,Fake Love,Drake,https://open.spotify.com/track/343YBumqHu19cGo...
8248,Starboy,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...
8249,Closer,"The Chainsmokers, Halsey",https://open.spotify.com/track/7BKLCZ1jbUBVqRi...
8250,Black Beatles,"Rae Sremmurd, Gucci Mane",https://open.spotify.com/track/6fujklziTHa8uoM...


In [13]:
print('total artists:',len(data_usa['artist'].tolist()),'; total songs (with unique names):',len(data_usa['title'].tolist()),
    '; unique artists:',len(data_usa['artist'].unique().tolist()),'; unique songs (with unique names):',len(data_usa['title'].unique().tolist())
)

total artists: 455057 ; total songs (with unique names): 455057 unique artists: 5213 ; unique songs (with unique names): 11842


In [16]:
data_usa.drop_duplicates(inplace=True)
print(data_usa.shape)
data_usa.head()

(14663, 3)


,title,artist,url
8246,Bad and Boujee (feat. Lil Uzi Vert),Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...
8247,Fake Love,Drake,https://open.spotify.com/track/343YBumqHu19cGo...
8248,Starboy,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...
8249,Closer,"The Chainsmokers, Halsey",https://open.spotify.com/track/7BKLCZ1jbUBVqRi...
8250,Black Beatles,"Rae Sremmurd, Gucci Mane",https://open.spotify.com/track/6fujklziTHa8uoM...


In [17]:
print('unique artists:',len(data_usa['artist'].unique().tolist()),'; unique songs (with unique names):',len(data_usa['title'].unique().tolist()))

unique artists: 5213 ; unique songs (with unique names): 11842


making all `artist` lowercase, removing all spaces and weird characters

In [20]:
data_usa['artist_clean'] = data_usa['artist'].str.strip().str.lower()
print(len(data_usa['artist_clean'].tolist()), len(data_usa['artist'].unique().tolist()))

14663 5213


doing the same with the `tracks`

In [21]:
data_usa['track_clean'] = data_usa['title'].str.strip().str.lower()
print(len(data_usa['track_clean'].unique().tolist()), len(data_usa['title'].unique().tolist()))

11635 11842


Fixing the `artist`s, in order to remove duplicated artist or typos -- replacing by the most common occurance by similarity.

In [22]:
def parecido_artistas(col, dict_artistas):   
    maximo = 0
    for key in dict_artistas.keys():
    #print(key)
        #print(genero)
        comparo = fuzz.ratio(key, col)
        if comparo > maximo:
            maximo = comparo
            resultado = key
    if maximo > 90:
        return resultado
    else:
        return col

In [26]:
data_usa['artist_clean'].value_counts()

taylor swift                 200
drake                        143
juice wrld                   123
future                       116
trippie redd                  97
                            ... 
frightened rabbit              1
breathe carolina, dropgun      1
brandon wardell                1
raye, mabel, stefflon don      1
cooper alan, rvshvd            1
Name: artist_clean, Length: 5211, dtype: int64

In [28]:
data_usa[data_usa['artist_clean'].str.contains('drake') == True]['artist_clean'].unique().tolist()[:10]

['drake',
 'drake, wizkid, kyla',
 'drake, 21 savage',
 'drake, rihanna',
 'rihanna, drake',
 'drake, future',
 'migos, drake',
 'drake, quavo, travis scott',
 'future, drake, young thug',
 'lil baby, drake']

In [29]:
fuzz.ratio(
    'drake, future',
    'future, drake, young thug'
    )

52.63157894736843

In [30]:
fuzz.ratio(
    'drake, rihanna',
    'rihanna, drake'
    )

50.0

In [31]:
cuenta_artistas = Counter(artistas for artistas in data_usa['artist_clean'])
print(len(cuenta_artistas))
cuenta_artistas

5211


Counter({'migos': 67,
         'drake': 143,
         'the weeknd, daft punk': 3,
         'the chainsmokers, halsey': 1,
         'rae sremmurd, gucci mane': 1,
         'shelley fka dram': 4,
         'drake, wizkid, kyla': 2,
         'aminé': 22,
         'dj snake, justin bieber': 2,
         'big sean': 36,
         'bruno mars': 8,
         'machine gun kelly': 42,
         '21 savage, metro boomin': 19,
         'zayn, taylor swift': 1,
         'maroon 5, kendrick lamar': 1,
         'rob $tone': 1,
         'j. cole': 48,
         'young m.a': 4,
         'the weeknd': 56,
         'chance the rapper': 19,
         'hailee steinfeld, grey, zedd': 2,
         'the chainsmokers': 24,
         'ariana grande, nicki minaj': 2,
         'shawn mendes': 45,
         'martin garrix, bebe rexha': 1,
         'drake, 21 savage': 1,
         'justin timberlake': 23,
         'fifth harmony': 4,
         'twenty one pilots': 35,
         'lil uzi vert': 65,
         'ariana grande': 71,

In [32]:
dict_artistas = dict(cuenta_artistas)
dict_artistas = {k:v for k,v in dict_artistas.items() if v > 1}
print(len(dict_artistas))
dict_artistas#.get(3)

1510


{'migos': 67,
 'drake': 143,
 'the weeknd, daft punk': 3,
 'shelley fka dram': 4,
 'drake, wizkid, kyla': 2,
 'aminé': 22,
 'dj snake, justin bieber': 2,
 'big sean': 36,
 'bruno mars': 8,
 'machine gun kelly': 42,
 '21 savage, metro boomin': 19,
 'j. cole': 48,
 'young m.a': 4,
 'the weeknd': 56,
 'chance the rapper': 19,
 'hailee steinfeld, grey, zedd': 2,
 'the chainsmokers': 24,
 'ariana grande, nicki minaj': 2,
 'shawn mendes': 45,
 'justin timberlake': 23,
 'fifth harmony': 4,
 'twenty one pilots': 35,
 'lil uzi vert': 65,
 'ariana grande': 71,
 'calvin harris': 13,
 'james arthur': 7,
 'kyle': 11,
 'sia': 12,
 'rihanna': 4,
 'major lazer': 5,
 'desiigner': 6,
 'alessia cara': 7,
 'starley': 2,
 'rae sremmurd': 6,
 'adele': 31,
 'john legend': 10,
 'justin bieber': 53,
 'drake, future': 2,
 'the weeknd, kendrick lamar': 2,
 'the jackson 5': 7,
 'future': 116,
 'travis scott': 31,
 'childish gambino': 19,
 'michael jackson': 6,
 'clean bandit': 6,
 'marvin gaye, tammi terrell': 4,

In [33]:
data_test = data_usa.sample(1000)

In [34]:
data_test['artist'] = data_test.apply(lambda x: parecido_artistas(x['artist_clean'], dict_artistas), axis=1)
data_test[['artist_clean','artist']].value_counts()

artist_clean                            artist                                
drake                                   drake                                     21
taylor swift                            taylor swift                              16
tory lanez                              tory lanez                                14
the weeknd                              the weeknd                                 8
a boogie wit da hoodie                  a boogie wit da hoodie                     7
                                                                                  ..
imarkkeyz                               imarkkeyz                                  1
inverness                               inverness                                  1
isaiah rashad                           isaiah rashad                              1
issam alnajjar, loud luxury, ali gatie  issam alnajjar, loud luxury, ali gatie     1
zedd, maren morris, grey                zedd, maren morris, grey       

In [35]:
tqdm.pandas()

data_usa['artist_clean_new'] = data_usa.progress_apply(lambda x: parecido_artistas(x['artist_clean'], dict_artistas), axis=1)
data_usa[['artist_clean','artist_clean_new']].value_counts()

100%|██████████| 14663/14663 [00:09<00:00, 1601.17it/s]


artist_clean                                       artist_clean_new                                 
taylor swift                                       taylor swift                                         200
drake                                              drake                                                143
juice wrld                                         juice wrld                                           123
future                                             future                                               116
trippie redd                                       trippie redd                                          97
                                                                                                       ... 
ilovemakonnen                                      ilovemakonnen                                          1
ilytommy, savage ga$p                              ilytommy, savage ga$p                                  1
imad royal                         

In [36]:
data_usa['check'] = np.where(data_usa['artist_clean_new'] == data_usa['artist_clean'], 'same', 'different' )
data_usa['check'].value_counts()

same         14659
different        4
Name: check, dtype: int64

In [37]:
data_usa[data_usa['check']== 'different']

,title,artist,url,artist_clean,track_clean,artist_clean_new,check
12579761,Eye 2 Eye,"Huncho Jack, Travis Scott, Quavo, Takeoff",https://open.spotify.com/track/4vwfOiyWxXeEdx4...,"huncho jack, travis scott, quavo, takeoff",eye 2 eye,"huncho jack, travis scott, quavo, offset",different
14418350,Highwomen,"The Highwomen, Brandi Carlile, Natalie Hemby, ...",https://open.spotify.com/track/0CTNLJMN9dMG4cl...,"the highwomen, brandi carlile, natalie hemby, ...",highwomen,"the highwomen, brandi carlile, natalie hemby, ...",different
24561954,Raid,"Madvillain, Madlib, MF DOOM, MED",https://open.spotify.com/track/1P3eYnB5KiF4S2q...,"madvillain, madlib, mf doom, med",raid,"madvillain, madlib, mf doom",different
25557711,Want me,Cl4ppers,https://open.spotify.com/track/1sAcYqiUbqZrV7v...,cl4ppers,want me,cl4pers,different


Doing the same process with the `tracks`, in order to remove typos.

In [38]:
data_usa.head()

,title,artist,url,artist_clean,track_clean,artist_clean_new,check
8246,Bad and Boujee (feat. Lil Uzi Vert),Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),migos,same
8247,Fake Love,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,drake,same
8248,Starboy,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy,"the weeknd, daft punk",same
8249,Closer,"The Chainsmokers, Halsey",https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer,"the chainsmokers, halsey",same
8250,Black Beatles,"Rae Sremmurd, Gucci Mane",https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles,"rae sremmurd, gucci mane",same


In [39]:
data_usa['track_clean'].value_counts()

intro                                               14
better                                              11
sleigh ride                                         11
body                                                 9
heaven                                               8
                                                    ..
go off (with lil uzi vert, quavo & travis scott)     1
why are you here                                     1
side                                                 1
diamonds (feat. gunna)                               1
vapors                                               1
Name: track_clean, Length: 11635, dtype: int64

In [40]:
data_usa[data_usa['track_clean'].str.contains("atrévete") == True]['track_clean'].unique().tolist()[:10]

[]

In [41]:
fuzz.ratio(
    'atrévete-te-te', 'atrévete') ## the fuzz ratio needs to be bigger than this value

72.72727272727273

In [42]:
cuenta_tracks = Counter(track for track in data_usa['track_clean'])
print(len(cuenta_tracks))
cuenta_tracks

11635


Counter({'bad and boujee (feat. lil uzi vert)': 2,
         'fake love': 2,
         'starboy': 2,
         'closer': 4,
         'black beatles': 1,
         'broccoli (feat. lil yachty)': 1,
         'one dance': 2,
         'caroline': 3,
         'let me love you': 2,
         'bounce back': 1,
         'i feel it coming': 1,
         '24k magic': 1,
         'bad things (with camila cabello)': 3,
         'x (feat. future)': 1,
         'i don’t wanna live forever (fifty shades darker) - from "fifty shades darker (original motion picture soundtrack)"': 1,
         "don't wanna know": 2,
         'chill bill (feat. j. davi$ & spooks)': 1,
         'deja vu': 4,
         'ooouuu': 1,
         'party monster': 1,
         'no problem (feat. lil wayne & 2 chainz)': 1,
         'no heart': 1,
         'starving': 2,
         "don't let me down (feat. daya)": 1,
         'side to side': 2,
         'treat you better': 3,
         'in the name of love': 1,
         'sneakin’': 1,
       

In [43]:
dict_tracks = dict(cuenta_tracks)
dict_tracks = {k:v for k,v in dict_tracks.items() if v > 4}
print(len(dict_tracks))
dict_tracks

77


{'immortal': 5,
 'all night': 6,
 'water': 6,
 'sorry': 5,
 'famous': 5,
 'my way': 5,
 'mine': 5,
 'friends': 7,
 'heaven': 8,
 'thunder': 6,
 'rubbin off the paint': 6,
 'unforgettable': 6,
 'believer': 6,
 'what lovers do (feat. sza)': 5,
 'attention': 5,
 'issues': 7,
 'wait': 5,
 'hurricane': 5,
 'down': 5,
 'changes': 7,
 'eyes closed': 6,
 'stay': 5,
 'intro': 14,
 'pain': 5,
 'sixteen': 7,
 'circles': 6,
 'sunflower - spider-man: into the spider-verse': 5,
 'lucid dreams': 5,
 'all girls are the same': 5,
 'truth hurts': 5,
 'moonlight': 8,
 'california': 5,
 'dreams': 6,
 'alone': 6,
 'numb': 8,
 'whatever it takes': 5,
 'mamacita': 5,
 'paradise': 6,
 'toxic': 7,
 'why': 7,
 'kings & queens': 6,
 'body': 9,
 'sober': 6,
 'yeah right': 5,
 'better': 11,
 'honey': 6,
 'jingle bell rock': 6,
 'money': 5,
 'have yourself a merry little christmas': 6,
 'winter wonderland': 5,
 'sleigh ride': 11,
 'christmas (baby please come home)': 8,
 'my only wish (this year)': 5,
 'this is hal

In [44]:
def parecido_tracks(col, dict_tracks):   
    maximo = 0
    for key in dict_tracks.keys():
    #print(key)
        #print(genero)
        comparo = fuzz.ratio(key, col)
        if comparo > maximo:
            maximo = comparo
            resultado = key
    if maximo > 90:
        return resultado
    else:
        return col

In [45]:
tqdm.pandas()

data_usa['track_clean_new'] = data_usa.progress_apply(lambda x: parecido_tracks(x['track_clean'], dict_tracks), axis=1)
data_usa[['track_clean','track_clean_new']].value_counts()

100%|██████████| 14663/14663 [00:00<00:00, 25319.16it/s]


track_clean                 track_clean_new           
intro                       intro                         14
better                      better                        11
sleigh ride                 sleigh ride                   11
body                        body                           9
moonlight                   moonlight                      8
                                                          ..
blindfold (feat. lil baby)  blindfold (feat. lil baby)     1
jacques (with ‪tove lo)     jacques (with ‪tove lo)        1
blind (feat. young thug)    blind (feat. young thug)       1
blind                       blind                          1
항상 (hangsang)               항상 (hangsang)                  1
Length: 11635, dtype: int64

In [46]:
data_usa['check'] = np.where(data_usa['track_clean_new'] == data_usa['track_clean'], 'same', 'different')
data_usa['check'].value_counts()

same         14652
different       11
Name: check, dtype: int64

In [47]:
data_usa[data_usa['check'] == 'different']

,title,artist,url,artist_clean,track_clean,artist_clean_new,check,track_clean_new
8341,Change,J. Cole,https://open.spotify.com/track/3pjUyVbFmM96tYh...,j. cole,change,j. cole,different,changes
8006582,Change,NF,https://open.spotify.com/track/4cBlIM5AjnQ74f5...,nf,change,nf,different,changes
8295136,ATTENTION!,XXXTENTACION,https://open.spotify.com/track/0RjJSBBEolCDJ6w...,xxxtentacion,attention!,xxxtentacion,different,attention
11031122,All Me,"Drake, 2 Chainz, Big Sean",https://open.spotify.com/track/4kNvYhyl8R6m1vy...,"drake, 2 chainz, big sean",all me,"drake, 2 chainz, big sean",different,call me
12466147,Believe,Eminem,https://open.spotify.com/track/2XDhuJvvx2wcIRO...,eminem,believe,eminem,different,believer
12790687,Believe,"A-Trak, Quavo, Lil Yachty",https://open.spotify.com/track/4CQGG4DimjEDPEl...,"a-trak, quavo, lil yachty",believe,"a-trak, quavo, lil yachty",different,believer
14167429,Runway,Stunna Girl,https://open.spotify.com/track/0BjEQJZy4rfUNpK...,stunna girl,runway,stunna girl,different,runaway
15679455,Dream,Shawn Mendes,https://open.spotify.com/track/0bZFa5DZcx9HWx2...,shawn mendes,dream,shawn mendes,different,dreams
24433919,Run Away,Feki,https://open.spotify.com/track/3MV2ErW6oARiFoX...,feki,run away,feki,different,runaway
25058846,Want me!,Cl4pers,https://open.spotify.com/track/5ZeCEofrCTYS7Zw...,cl4pers,want me!,cl4pers,different,want me


In [48]:
print(len(data_usa['track_clean_new'].unique().tolist()), len(data_usa['title'].unique().tolist()))

11627 11842


# Conversion
I will save the input file as .pkl as the original one is too big for github.

In [49]:
data_final = data_usa.drop_duplicates(subset=['artist_clean_new', 'track_clean_new', 'url'])
print(data_final.shape)
data_final.head()

(14659, 8)


,title,artist,url,artist_clean,track_clean,artist_clean_new,check,track_clean_new
8246,Bad and Boujee (feat. Lil Uzi Vert),Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),migos,same,bad and boujee (feat. lil uzi vert)
8247,Fake Love,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,drake,same,fake love
8248,Starboy,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy,"the weeknd, daft punk",same,starboy
8249,Closer,"The Chainsmokers, Halsey",https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer,"the chainsmokers, halsey",same,closer
8250,Black Beatles,"Rae Sremmurd, Gucci Mane",https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles,"rae sremmurd, gucci mane",same,black beatles


In [51]:
with open('../data/pickle/dataset_viral150_unique.pickle', 'wb') as data_nowplaying:
    pickle.dump(data_final[['url', 'artist_clean', 'track_clean']], data_nowplaying)